In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_3_Bin_Packing/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-21 16:00:28.226006


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

This problem is a classic example of a bin packing problem, which is a type of optimization problem. Here's how you can formulate it as a mathematical optimization model:

Parameters:
- Let W be the set of weights for each item, where W = {w₁, w₂, ..., wₙ}.
- Let C be the capacity of each box.
- Let B be the upper bound on the number of boxes.

Decision Variables:
- Let xᵢⱼ be a binary decision variable that equals 1 if item i is packed in box j, and 0 otherwise.

Objective Function:
The objective is to minimize the number of boxes used. This can be represented as:
Minimize Σ  yⱼ
Where yⱼ is a binary variable that equals 1 if box j is used, and 0 otherwise.

Constraints:
1. Each item is packed into exactly one box:
Σ xᵢⱼ = 1 for all i

2. The total weight of items in a box does not exceed the box's capacity:
Σ wᵢ * xᵢⱼ ≤ C * yⱼ for all j

3. The number of boxes used does not exceed the upper bound:
Σ yⱼ ≤ B

This model is a mixed integer linear programming (MILP) problem. The decision variables xᵢⱼ and yⱼ are binary, meaning they can only take on the values of 0 or 1. The objective function and constraints are linear, making this model a MILP problem. This problem is NP-hard, meaning that it is computationally difficult to solve, especially for large instances. However, there are many algorithms and solvers available to find optimal or near-optimal solutions.

In [6]:
print(response_1_text)

This problem is a classic example of a bin packing problem, which is a type of optimization problem. Here's how you can formulate it as a mathematical optimization model:

Parameters:
- Let W be the set of weights for each item, where W = {w₁, w₂, ..., wₙ}.
- Let C be the capacity of each box.
- Let B be the upper bound on the number of boxes.

Decision Variables:
- Let xᵢⱼ be a binary decision variable that equals 1 if item i is packed in box j, and 0 otherwise.

Objective Function:
The objective is to minimize the number of boxes used. This can be represented as:
Minimize Σ  yⱼ
Where yⱼ is a binary variable that equals 1 if box j is used, and 0 otherwise.

Constraints:
1. Each item is packed into exactly one box:
Σ xᵢⱼ = 1 for all i

2. The total weight of items in a box does not exceed the box's capacity:
Σ wᵢ * xᵢⱼ ≤ C * yⱼ for all j

3. The number of boxes used does not exceed the upper bound:
Σ yⱼ ≤ B

This model is a mixed integer linear programming (MILP) problem. The decision 

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here is a Python Pyomo code for the bin packing problem described:

```python
from pyomo.environ import *

# Sample data
W = [5, 10, 15, 20, 25]  # weights for each item
C = 30  # capacity of each box
B = 3  # upper bound on the number of boxes
n = len(W)  # number of items

# Initialize the model
model = ConcreteModel()

# Define the decision variables
model.x = Var(range(n), range(B), domain=Binary)
model.y = Var(range(B), domain=Binary)

# Define the objective function
model.obj = Objective(expr=sum(model.y[j] for j in range(B)), sense=minimize)

# Define the constraints
model.c1 = ConstraintList()
model.c2 = ConstraintList()
model.c3 = Constraint()

for i in range(n):
    model.c1.add(sum(model.x[i, j] for j in range(B)) == 1)

for j in range(B):
    model.c2.add(sum(W[i] * model.x[i, j] for i in range(n)) <= C * model.y[j])

model.c3 = sum(model.y[j] for j in range(B)) <= B

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", model.obj())
for i in range(n):
    for j in range(B):
        if model.x[i, j]() == 1:
            print(f"Item {i+1} is packed in box {j+1}")
```

This code uses the sample data for the weights of the items, the capacity of the boxes, and the upper bound on the number of boxes. You can replace these with your actual data. The code also uses the GLPK (GNU Linear Programming Kit) solver to solve the problem. You can replace 'glpk' with the name of the solver you want to use.

In [10]:
print(response_2_text)

Here is a Python Pyomo code for the bin packing problem described:

```python
from pyomo.environ import *

# Sample data
W = [5, 10, 15, 20, 25]  # weights for each item
C = 30  # capacity of each box
B = 3  # upper bound on the number of boxes
n = len(W)  # number of items

# Initialize the model
model = ConcreteModel()

# Define the decision variables
model.x = Var(range(n), range(B), domain=Binary)
model.y = Var(range(B), domain=Binary)

# Define the objective function
model.obj = Objective(expr=sum(model.y[j] for j in range(B)), sense=minimize)

# Define the constraints
model.c1 = ConstraintList()
model.c2 = ConstraintList()
model.c3 = Constraint()

for i in range(n):
    model.c1.add(sum(model.x[i, j] for j in range(B)) == 1)

for j in range(B):
    model.c2.add(sum(W[i] * model.x[i, j] for i in range(n)) <= C * model.y[j])

model.c3 = sum(model.y[j] for j in range(B)) <= B

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("

### Code Executability

In [11]:
from pyomo.environ import *

# Sample data
W = [5, 10, 15, 20, 25]  # weights for each item
C = 30  # capacity of each box
B = 3  # upper bound on the number of boxes
n = len(W)  # number of items

# Initialize the model
model = ConcreteModel()

# Define the decision variables
model.x = Var(range(n), range(B), domain=Binary)
model.y = Var(range(B), domain=Binary)

# Define the objective function
model.obj = Objective(expr=sum(model.y[j] for j in range(B)), sense=minimize)

# Define the constraints
model.c1 = ConstraintList()
model.c2 = ConstraintList()
model.c3 = Constraint()

for i in range(n):
    model.c1.add(sum(model.x[i, j] for j in range(B)) == 1)

for j in range(B):
    model.c2.add(sum(W[i] * model.x[i, j] for i in range(n)) <= C * model.y[j])

model.c3 = sum(model.y[j] for j in range(B)) <= B

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", model.obj())
for i in range(n):
    for j in range(B):
        if model.x[i, j]() == 1:
            print(f"Item {i+1} is packed in box {j+1}")

Objective value: 3.0
Item 1 is packed in box 2
Item 2 is packed in box 3
Item 3 is packed in box 1
Item 4 is packed in box 3
Item 5 is packed in box 2


### Solution Correctness

In [15]:
from pyomo.environ import *


B = 9
w = [2,3,4,5,6,7,8]
q = [4,2,6,6,2,2,2]
s=[]
for j in range(len(w)):
    for i in range(q[j]):
        s.append(w[j])

def FFD(s, B):
    remain = [B]
    sol = [[]]
    for item in sorted(s, reverse=True):
        for j,free in enumerate(remain):
            if free >= item:
                remain[j] -= item
                sol[j].append(item)
                break
        else:
            sol.append([item])
            remain.append(B-item)
    return sol

n = len(s)
U = len(FFD(s, B))

# Sample data (replace with your own data)
W = {i : v for i, v in enumerate(s)} # Weight of items
C = 9  # Capacity of boxes
B = U  # Upper bound on the number of boxes used
n = len(W)  # number of items

# Initialize the model
model = ConcreteModel()

# Define the decision variables
model.x = Var(range(n), range(B), domain=Binary)
model.y = Var(range(B), domain=Binary)

# Define the objective function
model.obj = Objective(expr=sum(model.y[j] for j in range(B)), sense=minimize)

# Define the constraints
model.c1 = ConstraintList()
model.c2 = ConstraintList()
model.c3 = Constraint()

for i in range(n):
    model.c1.add(sum(model.x[i, j] for j in range(B)) == 1)

for j in range(B):
    model.c2.add(sum(W[i] * model.x[i, j] for i in range(n)) <= C * model.y[j])

model.c3 = sum(model.y[j] for j in range(B)) <= B

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", model.obj())
for i in range(n):
    for j in range(B):
        if model.x[i, j]() == 1:
            print(f"Item {i+1} is packed in box {j+1}")

Objective value: 13.0
Item 1 is packed in box 12
Item 2 is packed in box 10
Item 3 is packed in box 13
Item 4 is packed in box 13
Item 5 is packed in box 6
Item 6 is packed in box 3
Item 7 is packed in box 9
Item 8 is packed in box 1
Item 9 is packed in box 8
Item 10 is packed in box 7
Item 11 is packed in box 11
Item 12 is packed in box 13
Item 13 is packed in box 6
Item 14 is packed in box 9
Item 15 is packed in box 8
Item 16 is packed in box 1
Item 17 is packed in box 11
Item 18 is packed in box 7
Item 19 is packed in box 3
Item 20 is packed in box 4
Item 21 is packed in box 10
Item 22 is packed in box 12
Item 23 is packed in box 5
Item 24 is packed in box 2
